In [15]:
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import requests
import pandas as pd
import json

abq_coord = {
    'lat': 35.0844,
    'long': -106.6504
}

token = open(".mapbox_token").read()  # you need your own token


In [17]:
url = "https://coagisweb.cabq.gov/arcgis/rest/services/public/PublicArt/MapServer/0/query"

querystring = {"where": "1=1", "text": "", "objectIds": "", "time": "", "geometry": "", "geometryType": "esriGeometryEnvelope", "inSR": "", "spatialRel": "esriSpatialRelIntersects", "relationParam": "", "outFields": "*", "returnGeometry": "true", "maxAllowableOffset": "",
               "geometryPrecision": "", "outSR": "", "returnIdsOnly": "false", "returnCountOnly": "false", "orderByFields": "", "groupByFieldsForStatistics": "", "outStatistics": "", "returnZ": "false", "returnM": "false", "gdbVersion": "", "returnDistinctValues": "false", "f": "pjson"}

headers = {'user-agent': 'vscode-restclient'}

response = requests.request("GET", url, headers=headers, params=querystring)

art_list = response.json()['features']
arts = pd.json_normalize(art_list)
arts.columns = arts.columns.str.lstrip("attributes.")


In [18]:
url = "http://data.cabq.gov/transit/realtime/route/allroutes.json"

headers = {'user-agent': 'vscode-restclient'}

response = requests.request("GET", url, headers=headers)

busses = pd.DataFrame(response.json()['allroutes'])


In [34]:
bus_layer = go.Figure(go.Scattermapbox(
    mode="markers",
    customdata=np.stack(
        (busses['route_short_name'], busses['speed_mph'], busses['next_stop_name']), axis=-1),
    hovertemplate="<b>Route:</b> %{customdata[0]}<br><b>Speed:</b> %{customdata[1]} MPH<br><b>Next Stop:</b> %{customdata[2]}<extra></extra>",
    lon=busses['longitude'], lat=busses['latitude'],
    marker={'size': 7,
            'symbol': "bus"},
    text=busses['route_short_name'], textposition="bottom right"))

bus_layer.update_layout(
    margin={'l': 0, 'r': 0, 't': 0, 'b': 0},
    mapbox={
        'accesstoken': token,
        'center': {'lat': abq_coord['lat'],
                   'lon': abq_coord['long']},
        'style': 'outdoors', 'zoom': 11},
    showlegend=False)

bus_layer.show()


In [27]:
art_layer = go.Figure(go.Scattermapbox(
    mode="markers",
    customdata=np.stack(
        (arts['TITLE'], arts['ARTIST'], arts['TYPE'], arts['YEAR']), axis=-1),
    hovertemplate="<b>Title:</b> %{customdata[0]}<br><b>Artist:</b> %{customdata[1]}<br><b>Type:</b> %{customdata[2]}<br><b>Year:</b> %{customdata[3]}<extra></extra>",
    lon=arts['X'], lat=arts['Y'],
    marker={'size': 7,
            'symbol': "art-gallery"},
    text=arts['TITLE'], textposition="bottom right"))

art_layer.update_layout(
    margin={'l': 0, 'r': 0, 't': 0, 'b': 0},
    mapbox={
        'accesstoken': token,
        'center': {'lat': abq_coord['lat'],
                   'lon': abq_coord['long']},
        'style': 'outdoors', 'zoom': 11},
    showlegend=False)

art_layer.show()


In [37]:
bus_layer.add_scattermapbox(mode="markers",
                            customdata=np.stack(
                                (arts['TITLE'], arts['ARTIST'], arts['TYPE'], arts['YEAR']), axis=-1),
                            hovertemplate="<b>Title:</b> %{customdata[0]}<br><b>Artist:</b> %{customdata[1]}<br><b>Type:</b> %{customdata[2]}<br><b>Year:</b> %{customdata[3]}<extra></extra>",
                            lon=arts['X'], lat=arts['Y'],
                            marker={'size': 7,
                                    'symbol': "art-gallery"},
                            text=arts['TITLE'], textposition="bottom right")

bus_layer.show()


In [35]:
url = "https://coagisweb.cabq.gov/arcgis/rest/services/public/APD_Incidents/MapServer/0/query"

querystring = {"where":"1=1","text":"","objectIds":"","time":"","geometry":"","geometryType":"esriGeometryEnvelope","inSR":"","spatialRel":"esriSpatialRelIntersects","relationParam":"","outFields":"*","returnGeometry":"true","maxAllowableOffset":"","geometryPrecision":"","outSR":"","returnIdsOnly":"false","returnCountOnly":"false","orderByFields":"","groupByFieldsForStatistics":"","outStatistics":"","returnZ":"false","returnM":"false","gdbVersion":"","returnDistinctValues":"false","f":"pjson"}

headers = {'user-agent': 'vscode-restclient'}

response = requests.request("GET", url, headers=headers, params=querystring)

crime_incident_list = response.json()['features']
crime_incidents = pd.json_normalize(crime_incident_list)
crime_incidents.columns = crime_incidents.columns.str.lstrip("attributes.")

In [76]:
test = crime_incidents

In [77]:
def convert_to_lat_long(row):

    from pyproj import Proj, transform

    inProj = Proj('epsg:3857')
    outProj = Proj('epsg:4326')
    lat,long = transform(inProj, outProj, row['geometry.x'], row['geometry.y'])
    return pd.Series([lat,long])



In [80]:
crime_incidents[['lat', 'long']] = crime_incidents.apply(lambda row: convert_to_lat_long(row), axis=1)

C:\Users\austi\AppData\Local\Temp\ipykernel_2016\1648199641.py:7: DeprecationWarning:

This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1

C:\Users\austi\AppData\Local\Temp\ipykernel_2016\1648199641.py:7: DeprecationWarning:

This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1

C:\Users\austi\AppData\Local\Temp\ipykernel_2016\1648199641.py:7: DeprecationWarning:

This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1

C:\Users\austi\AppData\Local\Temp\ipykernel_2016\1648199641.py:7: DeprecationWarning:

This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1

C:\Users\austi\AppData\Local\Temp\ipykernel_2016\1648199641.py:7: DeprecationWarning:

This function is deprecated. See: https://pyproj4.github.io/pyproj/st

In [ ]:
bus_layer.add_scattermapbox(mode="markers",
                            customdata=np.stack(
                                (crime_incidents['IncidentType'], crime_incidents['ReportDateTime'], crime_incidents['BlockAddress']), axis=-1),
                            hovertemplate="<b>Incident:</b> %{customdata[0]}<br><b>Report Date:</b> %{customdata[1]}<br><b>Address:</b> %{customdata[2]}<br><extra></extra>",
                            lat=crime_incidents['lat'], lon=crime_incidents['long'],
                            marker={'size': 7,
                                    'symbol': "police"},
                            text=crime_incidents['IncidentType'], textposition="bottom right")

ValueError: Invalid property specified for object of type plotly.graph_objs.Scattermapbox: 'x'

Did you mean "ids"?

    Valid properties:
        below
            Determines if this scattermapbox trace's layers are to
            be inserted before the layer with the specified ID. By
            default, scattermapbox layers are inserted above all
            the base layers. To place the scattermapbox layers
            above every other layer, set `below` to "''".
        connectgaps
            Determines whether or not gaps (i.e. {nan} or missing
            values) in the provided data arrays are connected.
        customdata
            Assigns extra data each datum. This may be useful when
            listening to hover, click and selection events. Note
            that, "scatter" traces also appends customdata items in
            the markers DOM elements
        customdatasrc
            Sets the source reference on Chart Studio Cloud for
            `customdata`.
        fi